# YOLOv8 Custom Football Dataset Train and Predict

In [6]:
!rm -rf

In [7]:
%pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 749.6/749.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [8]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from shutil import copyfile
import matplotlib.pyplot as plt
from PIL import Image

import ultralytics
from ultralytics import YOLO
ultralytics.checks()

Ultralytics YOLOv8.1.45 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (AMD EPYC 7B12)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 28.9/107.7 GB disk)


# Check Annotation Data

In [9]:
## connecting to the google drive

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import os

# List the contents of /kaggle/input
print("Listing /kaggle/input:")
print(os.listdir("/content/drive/MyDrive/yolo/data"))

# List the contents of your specific dataset directory
dataset_path = "/content/drive/MyDrive/yolo/data"
print(f"Listing {dataset_path}:")
if os.path.exists(dataset_path):
    print(os.listdir(dataset_path))
else:
    print("Directory does not exist.")


Listing /kaggle/input:
['Copy of 6ty_f_50_we.jpg', 'Copy of 6ty_f_65_we.jpg', 'Copy of 6ty_f_65_we.txt', 'Copy of 6ty_f_90_we.txt', 'Copy of 6ty_f_90_we.jpg', 'Copy of 6ty_f_80_54r.txt', 'Copy of 6ty_f_80_54r.jpg', 'Copy of 6ty_f_85.jpg', 'Copy of 6ty_f_85.txt', 'Copy of 6ty_f_50_we.txt', 'Copy of 6ty_f_195_6ty.jpg', 'Copy of 6ty_f_110_54r.jpg', 'Copy of 6ty_f_130_22d.txt', 'Copy of 6ty_f_195_6ty.txt', 'Copy of 22d_f_15_rt3.jpg', 'Copy of 22d_f_25.jpg', 'Copy of 6ty_f_110_54r.txt', 'Copy of 22d_f_25.txt', 'Copy of 22d_f_15_rt3.txt', 'Copy of 6ty_f_130_22d.jpg', 'Copy of 22d_f_30_22d.jpg', 'Copy of 22d_f_85_fr.txt', 'Copy of 22d_f_85_fr.jpg', 'Copy of 22d_f_90.txt', 'Copy of 22d_f_55_22d.jpg', 'Copy of 22d_f_30_22d.txt', 'Copy of 22d_f_55_22d.txt', 'Copy of 22d_f_80.jpg', 'Copy of 22d_f_80.txt', 'Copy of 22d_f_90.jpg', 'Copy of 22d_f_125_fre.jpg', 'Copy of 22d_f_125.jpg', 'Copy of 22d_f_120_6ty.jpg', 'Copy of 22d_f_135_we.txt', 'Copy of 22d_f_165_fr.jpg', 'Copy of 22d_f_125_fre.txt', 'C

In [11]:
ipaths=[]
tpaths=[]
for dirname, _, filenames in os.walk('/content/drive/MyDrive/yolo/data'):
    for filename in filenames:
        if filename[-4:]=='.jpg':
            ipaths+=[os.path.join(dirname, filename)]
        elif filename[-4:]=='.txt':
            tpaths+=[os.path.join(dirname, filename)]
ipaths=sorted(ipaths)
tpaths=sorted(tpaths)
print(ipaths[0])
print(len(ipaths))

/content/drive/MyDrive/yolo/data/Copy of 22d_f_120_6ty.jpg
25


In [12]:
boxdata=[]
boxfile=[]
for i in range(len(tpaths)):
    file=tpaths[i]
    boxdata+=[np.loadtxt(file)]
    boxfile+=[file[0:-4].split('/')[-1]]
print(boxdata[0])

[[          0      0.2849     0.21019    0.021875    0.087963]
 [          0     0.41875     0.27546    0.017188    0.097222]
 [          0     0.32904     0.35486    0.022135     0.10694]
 [          0     0.38997     0.50671    0.032031    0.099537]
 [          0     0.42513      0.4794     0.03151    0.099537]
 [          0      0.4849     0.41412    0.034896    0.090278]
 [          0     0.48398     0.52523    0.036719    0.091204]
 [          0     0.47721     0.58102    0.048698    0.081481]
 [          1     0.64805     0.45532     0.01276    0.022685]
 [          0     0.56875     0.21736    0.019271    0.078241]
 [          0     0.58646     0.19745    0.021354    0.077315]
 [          0     0.68919     0.40833    0.033594    0.097222]
 [          0     0.95065     0.38518    0.029948     0.07963]]


In [13]:
BOX=pd.DataFrame()

for i in range(len(boxdata)):
    if type(boxdata[i][0])==np.float64:
        add=pd.DataFrame([boxdata[i]])
        add[5]=boxfile[i]
        BOX=pd.concat([BOX,add])
    else:
        add=pd.DataFrame(boxdata[i])
        add[5]=boxfile[i]
        BOX=pd.concat([BOX,add])

BOX2=BOX.reset_index(drop=True)
display(BOX2)

,0,1,2,3,4,5
0,0.0,0.284896,0.210185,0.021875,0.087963,Copy of 22d_f_120_6ty
1,0.0,0.418750,0.275463,0.017188,0.097222,Copy of 22d_f_120_6ty
2,0.0,0.329036,0.354861,0.022135,0.106944,Copy of 22d_f_120_6ty
3,0.0,0.389974,0.506713,0.032031,0.099537,Copy of 22d_f_120_6ty
4,0.0,0.425130,0.479398,0.031510,0.099537,Copy of 22d_f_120_6ty
...,...,...,...,...,...,...
302,0.0,0.353125,0.413657,0.026042,0.077315,Copy of 6ty_f_90_we
303,0.0,0.348698,0.471759,0.021354,0.096296,Copy of 6ty_f_90_we
304,0.0,0.407682,0.639120,0.036719,0.115278,Copy of 6ty_f_90_we
305,0.0,0.439193,0.629861,0.043490,0.124537,Copy of 6ty_f_90_we


In [14]:
class_map = {0:'player', 1:'football'}

In [15]:
for i in range(len(BOX2)):
    BOX2.iloc[i,0]=class_map[int(BOX2.iloc[i,0])]
display(BOX2)
display(BOX2.iloc[:,0].value_counts())

,0,1,2,3,4,5
0,player,0.284896,0.210185,0.021875,0.087963,Copy of 22d_f_120_6ty
1,player,0.418750,0.275463,0.017188,0.097222,Copy of 22d_f_120_6ty
2,player,0.329036,0.354861,0.022135,0.106944,Copy of 22d_f_120_6ty
3,player,0.389974,0.506713,0.032031,0.099537,Copy of 22d_f_120_6ty
4,player,0.425130,0.479398,0.031510,0.099537,Copy of 22d_f_120_6ty
...,...,...,...,...,...,...
302,player,0.353125,0.413657,0.026042,0.077315,Copy of 6ty_f_90_we
303,player,0.348698,0.471759,0.021354,0.096296,Copy of 6ty_f_90_we
304,player,0.407682,0.639120,0.036719,0.115278,Copy of 6ty_f_90_we
305,player,0.439193,0.629861,0.043490,0.124537,Copy of 6ty_f_90_we


0
player      291
football     16
Name: count, dtype: int64

In [16]:
def draw_box(n0):

    ipath=ipaths[n0]
    image=cv2.imread(ipath)
    H,W=image.shape[0],image.shape[1]
    file=ipath[0:-4].split('/')[-1]

    if BOX2[BOX2[5]==file] is not None:
        box=BOX2[BOX2[5]==file]
        box=box.reset_index(drop=True)

        for i in range(len(box)):
            label=box.iloc[i,0]
            x=box.iloc[i,1]
            y=box.iloc[i,2]
            w=box.iloc[i,3]
            h=box.iloc[i,4]
            x0=((x-w/2)*W).astype(int)
            y0=((y-h/2)*H).astype(int)
            x1=((x+w/2)*W).astype(int)
            y1=((y+h/2)*H).astype(int)
            if label=='player':
                cv2.rectangle(image,(x0,y0),(x1,y1),(0,255,0),5) #green
            elif label=='football':
                cv2.rectangle(image,(x0,y0),(x1,y1),(0,0,255),5) #blue

    #plt.imshow(image)
    #plt.show()

    return image

In [17]:
images1=[]
for i in tqdm(range(len(ipaths))):
    images1+=[draw_box(i)]

100%|██████████| 25/25 [00:11<00:00,  2.11it/s]


In [18]:
from matplotlib import animation, rc
rc('animation', html='jshtml')

In [19]:
def create_animation(ims):
    fig=plt.figure(figsize=(10,6))
    #plt.axis('off')
    im=plt.imshow(cv2.cvtColor(ims[0],cv2.COLOR_BGR2RGB))
    plt.close()
    def animate_func(i):
        im.set_array(cv2.cvtColor(ims[i],cv2.COLOR_BGR2RGB))
        return [im]
    return animation.FuncAnimation(fig, animate_func, frames=len(ims), interval=1000//2)

In [20]:
create_animation(images1)

Output hidden; open in https://colab.research.google.com to view.

# Split Train, Valid and Test

In [27]:
os.makedirs('/content/drive/MyDrive/yolo', exist_ok=True)
os.makedirs('/content/drive/MyDrive/yolo/train', exist_ok=True)
os.makedirs('/content/drive/MyDrive/yolo/valid', exist_ok=True)
os.makedirs('/content/drive/MyDrive/yolo/test', exist_ok=True)

In [28]:
for i in range(len(ipaths)):
    ipath=ipaths[i]
    ifile=ipath.split('/')[-1]
    tpath=tpaths[i]
    tfile=tpath.split('/')[-1]
    if i%3==0:
        copyfile(ipath, f'/content/drive/MyDrive/yolo/train/{ifile}')
        copyfile(tpath, f'/content/drive/MyDrive/yolo/train/{tfile}')
    elif i%3==1:
        copyfile(ipath, f'/content/drive/MyDrive/yolo/valid/{ifile}')
        copyfile(tpath, f'/content/drive/MyDrive/yolo/valid/{tfile}')
    else:
        copyfile(ipath, f'/content/drive/MyDrive/yolo/test/{ifile}')
        #copyfile(tpath, f'datasets/test/{tfile}')

# Create yaml file

In [29]:
import yaml

football_yaml = dict(
    train ='train',
    val ='valid',
    test='test',
    nc =2,
    names =['player','football']
)

with open('football.yaml', 'w') as outfile:
    yaml.dump(football_yaml, outfile, default_flow_style=True)

%cat football.yaml

{names: [player, football], nc: 2, test: test, train: train, val: valid}


# Train

In [30]:
model = YOLO("yolov8n.pt")

In [31]:
!yolo task=detect mode=train model=yolov8n.pt data=football.yaml epochs=20 imgsz=640

Ultralytics YOLOv8.1.45 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (AMD EPYC 7B12)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=football.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_wid

# Result of Training

In [33]:
tpaths2=[]
for dirname, _, filenames in os.walk('/content/drive/MyDrive/yolo/train'):
    for filename in filenames:
        if filename[-4:]=='.png' or filename[-4:]=='.jpg':
            tpaths2+=[(os.path.join(dirname, filename))]
tpaths2=sorted(tpaths2)
print(tpaths2[0])

/content/drive/MyDrive/yolo/train/Copy of 22d_f_120_6ty.jpg


In [34]:
for path in tpaths2:
    image = Image.open(path)
    image=np.array(image)
    plt.figure(figsize=(20,10))
    plt.imshow(image)
    plt.show()

Output hidden; open in https://colab.research.google.com to view.

# Predict

In [35]:
best_path0='runs/detect/train/weights/best.pt'
source0='/content/drive/MyDrive/yolo/test'

In [36]:
ppaths=[]
for dirname, _, filenames in os.walk(source0):
    for filename in filenames:
        if filename[-4:]=='.jpg':
            ppaths+=[(os.path.join(dirname, filename))]
ppaths=sorted(ppaths)
print(ppaths)

['/content/drive/MyDrive/yolo/test/Copy of 22d_f_125_fre.jpg', '/content/drive/MyDrive/yolo/test/Copy of 22d_f_165_fr.jpg', '/content/drive/MyDrive/yolo/test/Copy of 22d_f_200_rt3.jpg', '/content/drive/MyDrive/yolo/test/Copy of 22d_f_55_22d.jpg', '/content/drive/MyDrive/yolo/test/Copy of 22d_f_90.jpg', '/content/drive/MyDrive/yolo/test/Copy of 6ty_f_110_54r.jpg', '/content/drive/MyDrive/yolo/test/Copy of 6ty_f_50_we.jpg', '/content/drive/MyDrive/yolo/test/Copy of 6ty_f_85.jpg']


In [37]:
model2 = YOLO(best_path0)

In [38]:
!yolo task=detect mode=predict model={best_path0} conf=0.0062 source={source0}

Ultralytics YOLOv8.1.45 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (AMD EPYC 7B12)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs

image 1/8 /content/drive/MyDrive/yolo/test/Copy of 22d_f_125_fre.jpg: 384x640 27 players, 10 footballs, 105.8ms
image 2/8 /content/drive/MyDrive/yolo/test/Copy of 22d_f_165_fr.jpg: 384x640 48 players, 10 footballs, 93.6ms
image 3/8 /content/drive/MyDrive/yolo/test/Copy of 22d_f_200_rt3.jpg: 384x640 42 players, 13 footballs, 87.0ms
image 4/8 /content/drive/MyDrive/yolo/test/Copy of 22d_f_55_22d.jpg: 384x640 32 players, 15 footballs, 89.8ms
image 5/8 /content/drive/MyDrive/yolo/test/Copy of 22d_f_90.jpg: 384x640 33 players, 22 footballs, 90.0ms
image 6/8 /content/drive/MyDrive/yolo/test/Copy of 6ty_f_110_54r.jpg: 384x640 27 players, 11 footballs, 87.3ms
image 7/8 /content/drive/MyDrive/yolo/test/Copy of 6ty_f_50_we.jpg: 384x640 36 players, 13 footballs, 90.1ms
image 8/8 /content/drive/MyDrive/yolo/test/Copy of 6ty_f_85.jpg: 384x640

# Result of Prediction

In [39]:
results = model2.predict(source0,conf=0.0062)
print(len(results))


image 1/8 /content/drive/MyDrive/yolo/test/Copy of 22d_f_125_fre.jpg: 384x640 27 players, 10 footballs, 96.0ms
image 2/8 /content/drive/MyDrive/yolo/test/Copy of 22d_f_165_fr.jpg: 384x640 48 players, 10 footballs, 88.4ms
image 3/8 /content/drive/MyDrive/yolo/test/Copy of 22d_f_200_rt3.jpg: 384x640 42 players, 13 footballs, 87.9ms
image 4/8 /content/drive/MyDrive/yolo/test/Copy of 22d_f_55_22d.jpg: 384x640 32 players, 15 footballs, 89.7ms
image 5/8 /content/drive/MyDrive/yolo/test/Copy of 22d_f_90.jpg: 384x640 33 players, 22 footballs, 95.3ms
image 6/8 /content/drive/MyDrive/yolo/test/Copy of 6ty_f_110_54r.jpg: 384x640 27 players, 11 footballs, 90.1ms
image 7/8 /content/drive/MyDrive/yolo/test/Copy of 6ty_f_50_we.jpg: 384x640 36 players, 13 footballs, 91.1ms
image 8/8 /content/drive/MyDrive/yolo/test/Copy of 6ty_f_85.jpg: 384x640 30 players, 25 footballs, 89.4ms
Speed: 2.2ms preprocess, 91.0ms inference, 206.9ms postprocess per image at shape (1, 3, 384, 640)
8


In [43]:
import pandas as pd

In [45]:
import pandas as pd

PBOX = pd.DataFrame()
for i, result in enumerate(results):
    # Convert boxes to numpy array format
    boxes = result.boxes.xyxy.numpy()  # Assuming this returns a numpy array of shape [N, 4]
    confidence = result.boxes.conf.numpy()  # Assuming this returns a numpy array
    classes = result.boxes.cls.numpy()  # Assuming this returns a numpy array

    # Create a DataFrame from the numpy arrays
    arri = pd.DataFrame(boxes, columns=['x', 'y', 'x2', 'y2'])
    arri['confidence'] = confidence
    arri['class'] = classes

    # Add file and index information
    path = ppaths[i]
    file = path.split('/')[-1]
    arri['file'] = file
    arri['i'] = i

    # Concatenate with the main DataFrame
    PBOX = pd.concat([PBOX, arri], ignore_index=True)

PBOX.columns = ['x', 'y', 'x2', 'y2', 'confidence', 'class', 'file', 'i']
display(PBOX)


,x,y,x2,y2,confidence,class,file,i
0,1541.165405,957.453186,1710.516724,1142.751709,0.008211,0.0,Copy of 22d_f_125_fre.jpg,0
1,1537.343384,1054.569580,1705.232788,1214.467896,0.008209,0.0,Copy of 22d_f_125_fre.jpg,0
2,1759.611328,1086.843140,1913.232422,1241.825073,0.008201,0.0,Copy of 22d_f_125_fre.jpg,0
3,1742.337524,925.368713,1908.669067,1136.755127,0.007993,0.0,Copy of 22d_f_125_fre.jpg,0
4,3464.447021,694.742676,3623.514404,879.545898,0.007430,0.0,Copy of 22d_f_125_fre.jpg,0
...,...,...,...,...,...,...,...,...
389,85.016418,713.821655,1954.178223,1983.564697,0.006237,1.0,Copy of 6ty_f_85.jpg,7
390,424.537903,140.648254,1826.133179,1399.369019,0.006223,1.0,Copy of 6ty_f_85.jpg,7
391,0.000000,453.444580,571.356750,1505.752808,0.006218,0.0,Copy of 6ty_f_85.jpg,7
392,2578.274658,387.314026,3831.971436,1726.106445,0.006214,1.0,Copy of 6ty_f_85.jpg,7


In [51]:
PBOX=pd.DataFrame(columns=range(6))
for i in range(len(results)):
    arri=pd.DataFrame(results[i].boxes.boxes).astype(float)
    path=ppaths[i]
    file=path.split('/')[-1]
    arri=arri.assign(file=file)
    arri=arri.assign(i=i)
    PBOX=pd.concat([PBOX,arri],axis=0)
PBOX.columns=['x','y','x2','y2','confidence','class','file','i',]
display(PBOX)

AttributeError: 'Boxes' object has no attribute 'boxes'. See valid attributes below.

    Manages detection boxes, providing easy access and manipulation of box coordinates, confidence scores, class
    identifiers, and optional tracking IDs. Supports multiple formats for box coordinates, including both absolute and
    normalized forms.

    Attributes:
        data (torch.Tensor): The raw tensor containing detection boxes and their associated data.
        orig_shape (tuple): The original image size as a tuple (height, width), used for normalization.
        is_track (bool): Indicates whether tracking IDs are included in the box data.

    Properties:
        xyxy (torch.Tensor | numpy.ndarray): Boxes in [x1, y1, x2, y2] format.
        conf (torch.Tensor | numpy.ndarray): Confidence scores for each box.
        cls (torch.Tensor | numpy.ndarray): Class labels for each box.
        id (torch.Tensor | numpy.ndarray, optional): Tracking IDs for each box, if available.
        xywh (tor

In [46]:
PBOX['class']=PBOX['class'].apply(lambda x: class_map[int(x)])
PBOX=PBOX.reset_index(drop=True)
display(PBOX)
display(PBOX['class'].value_counts())

,x,y,x2,y2,confidence,class,file,i
0,1541.165405,957.453186,1710.516724,1142.751709,0.008211,player,Copy of 22d_f_125_fre.jpg,0
1,1537.343384,1054.569580,1705.232788,1214.467896,0.008209,player,Copy of 22d_f_125_fre.jpg,0
2,1759.611328,1086.843140,1913.232422,1241.825073,0.008201,player,Copy of 22d_f_125_fre.jpg,0
3,1742.337524,925.368713,1908.669067,1136.755127,0.007993,player,Copy of 22d_f_125_fre.jpg,0
4,3464.447021,694.742676,3623.514404,879.545898,0.007430,player,Copy of 22d_f_125_fre.jpg,0
...,...,...,...,...,...,...,...,...
389,85.016418,713.821655,1954.178223,1983.564697,0.006237,football,Copy of 6ty_f_85.jpg,7
390,424.537903,140.648254,1826.133179,1399.369019,0.006223,football,Copy of 6ty_f_85.jpg,7
391,0.000000,453.444580,571.356750,1505.752808,0.006218,player,Copy of 6ty_f_85.jpg,7
392,2578.274658,387.314026,3831.971436,1726.106445,0.006214,football,Copy of 6ty_f_85.jpg,7


class
player      275
football    119
Name: count, dtype: int64

In [47]:
def draw_box2(n0):

    ipath=ppaths[n0]
    image=cv2.imread(ipath)
    H,W=image.shape[0],image.shape[1]
    file=ipath.split('/')[-1]

    if PBOX[PBOX['file']==file] is not None:
        box=PBOX[PBOX['file']==file]
        box=box.reset_index(drop=True)
        #display(box)

        for i in range(len(box)):
            label=box.iloc[i,5]
            x=int(box.iloc[i,0])
            y=int(box.iloc[i,1])
            x2=int(box.iloc[i,2])
            y2=int(box.iloc[i,3])
            #print(label,x,y,x2,y2)

            if label=='player':
                cv2.rectangle(image,(x,y),(x2,y2),(0,255,0),5) #green
            elif label=='football':
                cv2.rectangle(image,(x,y),(x2,y2),(0,0,255),5) #blue

    #plt.imshow(image)
    #plt.show()

    return image

In [48]:
images2=[]
for i in tqdm(range(len(ppaths))):
    images2+=[draw_box2(i)]

100%|██████████| 8/8 [00:00<00:00,  8.68it/s]


In [49]:
create_animation(images2)

Output hidden; open in https://colab.research.google.com to view.